# Sentinel-1 SLC Burst Search Examples

This notebook demonstrates how to search for Sentinel-1 SLC burst products using the phidown library's burst mode feature.

> **Note:** If you encounter errors about missing `burst_mode` parameter, please restart the kernel (**Kernel > Restart Kernel**) and run the cells again. This ensures the latest phidown version with burst mode support is loaded.

## What are Sentinel-1 Bursts?

Sentinel-1 SLC (Single Look Complex) bursts are individual radar acquisitions within a larger SLC product. Each burst represents a specific geographic area captured during the satellite's pass. Burst mode allows you to search and access these individual bursts rather than downloading entire SLC products.

## Features:
- Search for individual Sentinel-1 SLC bursts (available from August 2, 2024 onwards)
- Filter by burst-specific parameters (Burst ID, Swath, Polarization, etc.)
- Combine with standard filters (temporal, spatial, orbit direction)
- Access bursts from parent products without downloading the full scene

## Key Burst Parameters:
- **burst_id**: Unique identifier for a specific burst
- **absolute_burst_id**: Global unique burst identifier
- **swath_identifier**: Swath name (IW1, IW2, IW3 for Interferometric Wide, EW1-EW5 for Extra Wide)
- **parent_product_name**: Name of the parent SLC product
- **parent_product_type**: Type of parent product (IW_SLC__1S or EW_SLC__1S)
- **operational_mode**: Acquisition mode (IW or EW)
- **polarisation_channels**: Polarization (VV, VH, HH, HV)
- **platform_serial_identifier**: Sentinel-1 satellite (A or B)
- **relative_orbit_number**: Relative orbit number
- **datatake_id**: Datatake identifier

## Usage:
1. Set `burst_mode=True` in the query_by_filter method
2. Add burst-specific filters as needed
3. Execute the query to retrieve burst results

### Key Takeaways:

1. **Enable Burst Mode**: Always set `burst_mode=True` to search for individual bursts instead of full products
2. **Data Availability**: Burst data is available from August 2, 2024 onwards
3. **Combine Filters**: You can combine temporal, spatial, and burst-specific filters for targeted searches
4. **Swath Selection**: Use `swath_identifier` to select specific swaths (IW1, IW2, IW3 for IW mode)
5. **Polarization**: Filter by polarization channels (VV, VH, HH, HV) based on your application
6. **Orbit Parameters**: Use `orbit_direction` and `relative_orbit_number` for consistent time-series
7. **Parent Products**: Search bursts from specific parent products using `parent_product_name`

### Common Use Cases:

- **InSAR Time Series**: Search for specific burst IDs across multiple dates with consistent orbit parameters
- **Regional Analysis**: Use AOI + swath/polarization filters to get bursts covering your study area
- **Product Decomposition**: Retrieve all bursts from a parent SLC product for individual processing
- **Systematic Monitoring**: Filter by relative orbit number and burst ID for regular acquisitions

### Valid Parameter Values:

- **swath_identifier**: 'IW1', 'IW2', 'IW3', 'EW1', 'EW2', 'EW3', 'EW4', 'EW5'
- **parent_product_type**: 'IW_SLC__1S', 'EW_SLC__1S'
- **operational_mode**: 'IW', 'EW'
- **polarisation_channels**: 'VV', 'VH', 'HH', 'HV'
- **platform_serial_identifier**: 'A' (Sentinel-1A), 'B' (Sentinel-1B)
- **orbit_direction**: 'ASCENDING', 'DESCENDING'

For more information, visit the [Copernicus Data Space Ecosystem documentation](https://dataspace.copernicus.eu/)

In [1]:
# Auto-reload modules during development
%load_ext autoreload
%autoreload 2

In [2]:
"""
Import Required Libraries

Import the CopernicusDataSearcher class from phidown for burst searching.

Note: If you get an error about 'burst_mode' parameter not found, please:
1. Restart the kernel (Kernel > Restart Kernel)
2. Re-run this cell
This ensures the latest phidown version with burst mode support is loaded.
"""
from phidown.search import CopernicusDataSearcher
import pandas as pd

# Set pandas display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

# Verify burst mode is available
print('Checking if burst mode is available...')
import inspect
sig = inspect.signature(CopernicusDataSearcher.query_by_filter)
if 'burst_mode' in sig.parameters:
    print('✓ Burst mode is available!')
else:
    print('✗ Burst mode not found. Please restart the kernel and re-run this cell.')

/Users/roberto.delprete/Library/CloudStorage/OneDrive-ESA/Desktop/Repos/phidown/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Checking if burst mode is available...
✓ Burst mode is available!


## Example 1: Basic Burst Search with Temporal Filter

The simplest burst search uses only temporal filters. Set `burst_mode=True` to search the Bursts endpoint instead of the Products endpoint.

In [3]:
"""
Basic Burst Search

Search for Sentinel-1 SLC bursts within a specific time range.
Data is available from August 2, 2024 onwards.
"""
searcher = CopernicusDataSearcher()

# Configure search for bursts with temporal filter
searcher.query_by_filter(
    burst_mode=True,
    start_date='2024-08-01T00:00:00',
    end_date='2024-08-03T00:00:00',
    top=10,
    count=True
)

# Display the query URL
print(f'Query URL: {searcher._build_query()}\n')

# Execute the query
df = searcher.execute_query()
print(f'Number of results: {len(df)}')
print(f'Total available results: {searcher.num_results}')

# Display first few results
searcher.display_results(top_n=5)

Query URL: https://catalogue.dataspace.copernicus.eu/odata/v1/Bursts?$filter=ContentDate/Start ge 2024-08-01T00:00:00 and ContentDate/Start le 2024-08-03T00:00:00&$orderby=ContentDate/Start desc&$top=10&$count=true

Number of results: 10
Total available results: 58014


,Id,BurstId,SwathIdentifier,ParentProductName,PolarisationChannels,OrbitDirection,ContentDate
0,5094c40f-0010-4ff1-95d2-aa0ed30622ad,38900,IW3,S1A_IW_SLC__1SDV_20240802T234847_20240802T2349...,VH,ASCENDING,"{'Start': '2024-08-02T23:49:07.596418Z', 'End'..."
1,af330f8f-ac59-4e89-9ef7-4a587b7a5b06,38900,IW3,S1A_IW_SLC__1SDV_20240802T234847_20240802T2349...,VV,ASCENDING,"{'Start': '2024-08-02T23:49:07.596418Z', 'End'..."
2,8f93a6ad-0c0b-42cd-a108-1c7d78f1e46b,38900,IW2,S1A_IW_SLC__1SDV_20240802T234847_20240802T2349...,VH,ASCENDING,"{'Start': '2024-08-02T23:49:06.518381Z', 'End'..."
3,d38c2b58-941f-4ad4-99c3-4bddbb9e5ac1,38900,IW2,S1A_IW_SLC__1SDV_20240802T234847_20240802T2349...,VV,ASCENDING,"{'Start': '2024-08-02T23:49:06.518381Z', 'End'..."
4,f0d8dcc9-8af9-4ead-b9fe-4aa51d5f47de,38900,IW1,S1A_IW_SLC__1SDV_20240802T234847_20240802T2349...,VH,ASCENDING,"{'Start': '2024-08-02T23:49:05.686178Z', 'End'..."


## Example 2: Burst Search with Spatial Filter (Area of Interest)

You can combine burst mode with spatial filtering using a WKT polygon to define your area of interest.

In [4]:
"""
Burst Search with AOI

Search for bursts that intersect with a specific geographic area.
The AOI is defined as a WKT (Well-Known Text) polygon.
"""
# Define an area of interest (example: region in central Europe)
aoi_wkt = """POLYGON((12.655118166047592 47.44667197521409, 
                       21.39065656328509 48.347694733853245, 
                       28.334291357162826 41.877123516783655, 
                       17.47086198383573 40.35854475076158, 
                       12.655118166047592 47.44667197521409))"""

searcher = CopernicusDataSearcher()

# Configure search with AOI and temporal filter
searcher.query_by_filter(
    burst_mode=True,
    aoi_wkt=aoi_wkt,
    start_date='2024-08-01T00:00:00',
    end_date='2024-08-03T00:00:00',
    top=10,
    count=True
)

print(f'Query URL: {searcher._build_query()}\n')

# Execute and display results
df = searcher.execute_query()
print(f'Found {len(df)} bursts in the specified area')
searcher.display_results(top_n=5)

Query URL: https://catalogue.dataspace.copernicus.eu/odata/v1/Bursts?$filter=OData.CSC.Intersects(area=geography'SRID=4326;POLYGON((12.655118166047592 47.44667197521409, 21.39065656328509 48.347694733853245, 28.334291357162826 41.877123516783655, 17.47086198383573 40.35854475076158, 12.655118166047592 47.44667197521409))') and ContentDate/Start ge 2024-08-01T00:00:00 and ContentDate/Start le 2024-08-03T00:00:00&$orderby=ContentDate/Start desc&$top=10&$count=true

Found 10 bursts in the specified area


,Id,BurstId,SwathIdentifier,ParentProductName,PolarisationChannels,OrbitDirection,ContentDate
0,38af48ba-264e-4be5-b500-82355520cc2f,13720,IW3,S1A_IW_SLC__1SDV_20240802T043126_20240802T0431...,VH,DESCENDING,"{'Start': '2024-08-02T04:31:34.348445Z', 'End'..."
1,ad5eaf43-261b-4909-a45e-d3f6acb53f6b,13720,IW3,S1A_IW_SLC__1SDV_20240802T043126_20240802T0431...,VV,DESCENDING,"{'Start': '2024-08-02T04:31:34.348445Z', 'End'..."
2,93731915-d5ff-4566-93ea-fb49fd145dcc,13719,IW3,S1A_IW_SLC__1SDV_20240802T043126_20240802T0431...,VV,DESCENDING,"{'Start': '2024-08-02T04:31:31.590168Z', 'End'..."
3,dadbc607-73ba-4cf0-a69e-5c5264aa2faa,13719,IW3,S1A_IW_SLC__1SDV_20240802T043126_20240802T0431...,VH,DESCENDING,"{'Start': '2024-08-02T04:31:31.590168Z', 'End'..."
4,da72396d-7645-486c-9f7c-413d9939c1ef,13718,IW3,S1A_IW_SLC__1SDV_20240802T043126_20240802T0431...,VV,DESCENDING,"{'Start': '2024-08-02T04:31:28.831891Z', 'End'..."


## Example 3: Search by Specific Burst ID

If you know the Burst ID you're looking for, you can search for it directly.

In [5]:
"""
Search by Burst ID

Retrieve all acquisitions of a specific burst by its ID.
Burst IDs are integers that uniquely identify a burst location.
"""
searcher = CopernicusDataSearcher()

# Search for a specific burst ID
searcher.query_by_filter(
    burst_mode=True,
    burst_id=15804,  # Example burst ID
    start_date='2024-08-01T00:00:00',
    end_date='2024-08-15T00:00:00',
    top=20,
    count=True
)

print(f'Query URL: {searcher._build_query()}\n')

# Execute and display
df = searcher.execute_query()
print(f'Found {len(df)} acquisitions of burst {15804}')
searcher.display_results(top_n=5)

Query URL: https://catalogue.dataspace.copernicus.eu/odata/v1/Bursts?$filter=ContentDate/Start ge 2024-08-01T00:00:00 and ContentDate/Start le 2024-08-15T00:00:00 and BurstId eq 15804&$orderby=ContentDate/Start desc&$top=20&$count=true

Found 12 acquisitions of burst 15804


,Id,BurstId,SwathIdentifier,ParentProductName,PolarisationChannels,OrbitDirection,ContentDate
0,03e875f3-319a-4c8f-935b-685b2224721a,15804,IW3,S1A_IW_SLC__1SDV_20240814T060718_20240814T0607...,VH,DESCENDING,"{'Start': '2024-08-14T06:07:21.962547Z', 'End'..."
1,c5288339-81c9-4fd5-82cc-8764450b2a06,15804,IW3,S1A_IW_SLC__1SDV_20240814T060718_20240814T0607...,VV,DESCENDING,"{'Start': '2024-08-14T06:07:21.962547Z', 'End'..."
2,5d1ec25e-ca40-4f12-bd36-c291903ee40c,15804,IW2,S1A_IW_SLC__1SDV_20240814T060718_20240814T0607...,VV,DESCENDING,"{'Start': '2024-08-14T06:07:20.884510Z', 'End'..."
3,f739d767-f2e5-485b-957a-bdbc3421ee8b,15804,IW2,S1A_IW_SLC__1SDV_20240814T060718_20240814T0607...,VH,DESCENDING,"{'Start': '2024-08-14T06:07:20.884510Z', 'End'..."
4,3fe29409-4502-4cfb-8204-89b1f6a62adb,15804,IW1,S1A_IW_SLC__1SDV_20240814T060718_20240814T0607...,VH,DESCENDING,"{'Start': '2024-08-14T06:07:20.052307Z', 'End'..."


## Example 4: Search by Swath Identifier and Polarization

Filter bursts by specific swath (IW1, IW2, IW3) and polarization channel (VV, VH, HH, HV).

In [6]:
"""
Filter by Swath and Polarization

Search for bursts from a specific swath with specific polarization.
Swath identifiers: IW1, IW2, IW3 (Interferometric Wide) or EW1-EW5 (Extra Wide)
Polarization channels: VV, VH, HH, HV
"""
searcher = CopernicusDataSearcher()

# Search for IW2 swath with VV polarization
searcher.query_by_filter(
    burst_mode=True,
    swath_identifier='IW2',
    polarisation_channels='VV',
    start_date='2024-08-01T00:00:00',
    end_date='2024-08-03T00:00:00',
    top=10,
    count=True
)

print(f'Query URL: {searcher._build_query()}\n')

# Execute and display
df = searcher.execute_query()
print(f'Found {len(df)} bursts from swath IW2 with VV polarization')
searcher.display_results(top_n=5)

Query URL: https://catalogue.dataspace.copernicus.eu/odata/v1/Bursts?$filter=ContentDate/Start ge 2024-08-01T00:00:00 and ContentDate/Start le 2024-08-03T00:00:00 and SwathIdentifier eq 'IW2' and PolarisationChannels eq 'VV'&$orderby=ContentDate/Start desc&$top=10&$count=true

Found 10 bursts from swath IW2 with VV polarization


,Id,BurstId,SwathIdentifier,ParentProductName,PolarisationChannels,OrbitDirection,ContentDate
0,d38c2b58-941f-4ad4-99c3-4bddbb9e5ac1,38900,IW2,S1A_IW_SLC__1SDV_20240802T234847_20240802T2349...,VV,ASCENDING,"{'Start': '2024-08-02T23:49:06.518381Z', 'End'..."
1,cf6430ea-c4b8-4d3d-9232-07dbc5acd30b,38899,IW2,S1A_IW_SLC__1SDV_20240802T234847_20240802T2349...,VV,ASCENDING,"{'Start': '2024-08-02T23:49:03.760104Z', 'End'..."
2,763b7936-21ae-4b2d-b611-67a924a77045,38898,IW2,S1A_IW_SLC__1SDV_20240802T234847_20240802T2349...,VV,ASCENDING,"{'Start': '2024-08-02T23:49:01.001827Z', 'End'..."
3,042189e6-9054-4331-8be8-48c15ae3712b,38897,IW2,S1A_IW_SLC__1SDV_20240802T234847_20240802T2349...,VV,ASCENDING,"{'Start': '2024-08-02T23:48:58.243550Z', 'End'..."
4,1f0be57a-2733-4550-8a9e-532fd8c756dc,38896,IW2,S1A_IW_SLC__1SDV_20240802T234847_20240802T2349...,VV,ASCENDING,"{'Start': '2024-08-02T23:48:55.485273Z', 'End'..."


## Example 5: Search Bursts from a Specific Parent Product

If you know the parent SLC product name, you can retrieve all its constituent bursts.

In [7]:
"""
Search by Parent Product Name

Retrieve all bursts from a specific parent SLC product.
This is useful when you want to access individual bursts from a known product.
"""
searcher = CopernicusDataSearcher()

# Example parent product name (replace with an actual product name from your results)
parent_product = 'S1A_IW_SLC__1SDV_20240802T060719_20240802T060746_055030_06B44E_E7CC.SAFE'

searcher.query_by_filter(
    burst_mode=True,
    parent_product_name=parent_product,
    start_date='2024-08-01T00:00:00',
    end_date='2024-08-15T00:00:00',
    top=1000  # Get all bursts from this product
)

print(f'Query URL: {searcher._build_query()}\n')

# Execute and display
df = searcher.execute_query()
print(f'Found {len(df)} bursts in parent product')
print(f'Parent product: {parent_product}\n')
searcher.display_results(top_n=10)

Query URL: https://catalogue.dataspace.copernicus.eu/odata/v1/Bursts?$filter=ContentDate/Start ge 2024-08-01T00:00:00 and ContentDate/Start le 2024-08-15T00:00:00 and ParentProductName eq 'S1A_IW_SLC__1SDV_20240802T060719_20240802T060746_055030_06B44E_E7CC.SAFE'&$orderby=ContentDate/Start desc&$top=1000

Found 54 bursts in parent product
Parent product: S1A_IW_SLC__1SDV_20240802T060719_20240802T060746_055030_06B44E_E7CC.SAFE



,Id,BurstId,SwathIdentifier,ParentProductName,PolarisationChannels,OrbitDirection,ContentDate
0,beaedba3-52b2-4a42-a97c-6c40e56e6040,15812,IW3,S1A_IW_SLC__1SDV_20240802T060719_20240802T0607...,VH,DESCENDING,"{'Start': '2024-08-02T06:07:44.644714Z', 'End'..."
1,e7d3e00c-fb01-4c60-bcbb-5d5d41d4c627,15812,IW3,S1A_IW_SLC__1SDV_20240802T060719_20240802T0607...,VV,DESCENDING,"{'Start': '2024-08-02T06:07:44.644714Z', 'End'..."
2,a38752b8-cb26-4a4a-8e1d-4166a12ffa6c,15812,IW2,S1A_IW_SLC__1SDV_20240802T060719_20240802T0607...,VH,DESCENDING,"{'Start': '2024-08-02T06:07:43.566677Z', 'End'..."
3,fd1dce59-bbe6-48f4-9f98-a97a21b772e2,15812,IW2,S1A_IW_SLC__1SDV_20240802T060719_20240802T0607...,VV,DESCENDING,"{'Start': '2024-08-02T06:07:43.566677Z', 'End'..."
4,ea4cc725-f72e-47f7-a49e-b46bf3742eba,15812,IW1,S1A_IW_SLC__1SDV_20240802T060719_20240802T0607...,VH,DESCENDING,"{'Start': '2024-08-02T06:07:42.734474Z', 'End'..."
5,f8c4f619-b0e5-481e-afbe-b2fd8f244b4a,15812,IW1,S1A_IW_SLC__1SDV_20240802T060719_20240802T0607...,VV,DESCENDING,"{'Start': '2024-08-02T06:07:42.734474Z', 'End'..."
6,3d86f604-0c6c-4612-ba9c-42b08d3c33c7,15811,IW3,S1A_IW_SLC__1SDV_20240802T060719_20240802T0607...,VH,DESCENDING,"{'Start': '2024-08-02T06:07:41.886437Z', 'End'..."
7,c54482bd-44e2-4296-ac7f-809b304139a9,15811,IW3,S1A_IW_SLC__1SDV_20240802T060719_20240802T0607...,VV,DESCENDING,"{'Start': '2024-08-02T06:07:41.886437Z', 'End'..."
8,6fb4318c-f6ca-4130-92c3-548fa1bc1733,15811,IW2,S1A_IW_SLC__1SDV_20240802T060719_20240802T0607...,VH,DESCENDING,"{'Start': '2024-08-02T06:07:40.808400Z', 'End'..."
9,a60d901b-dbfd-445b-9e7b-46a4c35b9f5b,15811,IW2,S1A_IW_SLC__1SDV_20240802T060719_20240802T0607...,VV,DESCENDING,"{'Start': '2024-08-02T06:07:40.808400Z', 'End'..."


## Example 6: Filter by Orbit Direction and Relative Orbit Number

Combine orbit parameters with burst search to find specific orbital acquisitions.

In [8]:
"""
Filter by Orbit Parameters

Search for bursts from a specific orbit direction and relative orbit number.
This is useful for consistent time-series analysis.
"""
searcher = CopernicusDataSearcher()

searcher.query_by_filter(
    burst_mode=True,
    orbit_direction='DESCENDING',
    relative_orbit_number=8,
    operational_mode='IW',
    start_date='2024-08-01T00:00:00',
    end_date='2024-08-10T00:00:00',
    top=20,
    count=True
)

print(f'Query URL: {searcher._build_query()}\n')

# Execute and display
df = searcher.execute_query()
print(f'Found {len(df)} bursts from descending orbit #8')
searcher.display_results(top_n=5)

Query URL: https://catalogue.dataspace.copernicus.eu/odata/v1/Bursts?$filter=OrbitDirection eq 'DESCENDING' and ContentDate/Start ge 2024-08-01T00:00:00 and ContentDate/Start le 2024-08-10T00:00:00 and RelativeOrbitNumber eq 8 and OperationalMode eq 'IW'&$orderby=ContentDate/Start desc&$top=20&$count=true

Found 20 bursts from descending orbit #8


,Id,BurstId,SwathIdentifier,ParentProductName,PolarisationChannels,OrbitDirection,ContentDate
0,33566f0a-cad1-44a4-a050-8702d921de1e,15936,IW3,S1A_IW_SLC__1SDV_20240802T061309_20240802T0613...,VH,DESCENDING,"{'Start': '2024-08-02T06:13:26.671048Z', 'End'..."
1,f5a9eed2-1b5c-42ab-80e5-2453d028ca1f,15936,IW3,S1A_IW_SLC__1SDV_20240802T061309_20240802T0613...,VV,DESCENDING,"{'Start': '2024-08-02T06:13:26.671048Z', 'End'..."
2,9de9e560-d75e-4737-96c5-02adc5494d21,15936,IW2,S1A_IW_SLC__1SDV_20240802T061309_20240802T0613...,VH,DESCENDING,"{'Start': '2024-08-02T06:13:25.593011Z', 'End'..."
3,b6c882f9-66de-43ea-b058-4da6bb5f6bf1,15936,IW2,S1A_IW_SLC__1SDV_20240802T061309_20240802T0613...,VV,DESCENDING,"{'Start': '2024-08-02T06:13:25.593011Z', 'End'..."
4,680f349d-6d45-44ca-b556-da9e334006a8,15936,IW1,S1A_IW_SLC__1SDV_20240802T061309_20240802T0613...,VH,DESCENDING,"{'Start': '2024-08-02T06:13:24.760808Z', 'End'..."


## Example 7: Advanced Multi-Parameter Burst Search

Combine multiple burst-specific parameters for highly targeted searches.

In [9]:
"""
Advanced Multi-Parameter Search

Demonstrate combining multiple burst-specific filters for precise queries.
This example searches for a specific burst ID with additional constraints.
"""
searcher = CopernicusDataSearcher()

searcher.query_by_filter(
    burst_mode=True,
    burst_id=15804,
    swath_identifier='IW2',
    parent_product_type='IW_SLC__1S',
    orbit_direction='DESCENDING',
    relative_orbit_number=8,
    operational_mode='IW',
    polarisation_channels='VV',
    platform_serial_identifier='A',  # Sentinel-1A
    start_date='2024-08-01T00:00:00',
    end_date='2024-08-15T00:00:00',
    top=20,
    count=True
)

print(f'Query URL: {searcher._build_query()}\n')

# Execute and display
df = searcher.execute_query()
print(f'Found {len(df)} bursts matching all criteria')
print(f'Total available: {searcher.num_results}')
searcher.display_results(top_n=10)

Query URL: https://catalogue.dataspace.copernicus.eu/odata/v1/Bursts?$filter=OrbitDirection eq 'DESCENDING' and ContentDate/Start ge 2024-08-01T00:00:00 and ContentDate/Start le 2024-08-15T00:00:00 and BurstId eq 15804 and SwathIdentifier eq 'IW2' and ParentProductType eq 'IW_SLC__1S' and RelativeOrbitNumber eq 8 and OperationalMode eq 'IW' and PolarisationChannels eq 'VV' and PlatformSerialIdentifier eq 'A'&$orderby=ContentDate/Start desc&$top=20&$count=true

Found 2 bursts matching all criteria
Total available: 2


,Id,BurstId,SwathIdentifier,ParentProductName,PolarisationChannels,OrbitDirection,ContentDate
0,5d1ec25e-ca40-4f12-bd36-c291903ee40c,15804,IW2,S1A_IW_SLC__1SDV_20240814T060718_20240814T0607...,VV,DESCENDING,"{'Start': '2024-08-14T06:07:20.884510Z', 'End'..."
1,2353e9aa-a863-4048-b71f-a4028618334f,15804,IW2,S1A_IW_SLC__1SDV_20240802T060719_20240802T0607...,VV,DESCENDING,"{'Start': '2024-08-02T06:07:21.500461Z', 'End'..."


## Example 8: Analyzing Burst Results

Once you have burst results, you can analyze the data to understand burst distribution, coverage, and other characteristics.

In [10]:
"""
Analyze Burst Search Results

Perform basic analysis on burst search results to understand:
- Distribution across swaths
- Polarization channels available
- Parent products
- Temporal coverage
"""
searcher = CopernicusDataSearcher()

# Search for bursts with broad criteria
searcher.query_by_filter(
    burst_mode=True,
    orbit_direction='DESCENDING',
    swath_identifier='IW2',
    polarisation_channels='VV',
    start_date='2024-08-01T00:00:00',
    end_date='2024-08-03T00:00:00',
    top=100,
    count=True
)

# Execute query
df = searcher.execute_query()

print(f'=== Burst Search Results Analysis ===\n')
print(f'Total bursts found: {len(df)}')
print(f'Total available: {searcher.num_results}\n')

# Analyze burst distribution
if len(df) > 0:
    print(f'=== Burst Characteristics ===')
    
    # Check available columns
    if 'SwathIdentifier' in df.columns:
        print(f'\nSwath Distribution:')
        print(df['SwathIdentifier'].value_counts())
    
    if 'PolarisationChannels' in df.columns:
        print(f'\nPolarization Channels:')
        print(df['PolarisationChannels'].value_counts())
    
    if 'ParentProductName' in df.columns:
        print(f'\nNumber of unique parent products: {df["ParentProductName"].nunique()}')
    
    if 'BurstId' in df.columns:
        print(f'\nNumber of unique burst IDs: {df["BurstId"].nunique()}')
        print(f'Burst ID range: {df["BurstId"].min()} - {df["BurstId"].max()}')
    
    print(f'\n=== Sample Results ===')
    searcher.display_results(top_n=5)

=== Burst Search Results Analysis ===

Total bursts found: 100
Total available: 4369

=== Burst Characteristics ===

Swath Distribution:
SwathIdentifier
IW2    100
Name: count, dtype: int64

Polarization Channels:
PolarisationChannels
VV    100
Name: count, dtype: int64

Number of unique parent products: 12

Number of unique burst IDs: 100
Burst ID range: 35540 - 37561

=== Sample Results ===
